# VBA Benchmark Model

In [11]:
import numpy
import xlrd
from openpyxl import load_workbook
from vbaFunc import jumpoff, xTFR, clear_Input
from vbaFunc import readCNAPC, readSYI, readSATP, readERP, readTFR, readASFR, readLEMS, readMASMR
from vbaFunc import inputASFR, inputbirth, inputASDR, inputdeath

# Read in Excel Workbook (both Readable and Writable)
wt_loc = '../VBA_Benchmark/vba.xlsx'
rd_loc = '../VBA_Benchmark/original.xlsm'
wb_rd = xlrd.open_workbook(rd_loc)
wb_wt = load_workbook(wt_loc)

## xTRF

#### Generating Fertility Data

In [12]:
# Read in Sheets
sheet_label = wb_rd.sheet_by_name('Labels')
sheet_agesex = wb_rd.sheet_by_name('AgeSexERPs')
wb_wt_fertility = wb_wt["Fertility"]

# Define variables
numareas = int(sheet_label.cell_value(2256, 0))
numages = 18
sextypes = 2
set_year_female = 7
set_year_male = 5

# Create empty array for storing jumpoffERP and xTFR results
jumpoffERP = numpy.zeros((numareas, sextypes, numages))
jumpoffERP[:] = numpy.nan
result_xTFR = numpy.zeros((numareas))
result_xTFR[:] = numpy.nan

# Generate jumpoffERP for further calculation
jumpoffERP = jumpoff(numareas, numages, jumpoffERP, sheet_agesex, set_year_female, set_year_male)

# Generate xTFR vector for storing
result_xTFR = xTFR(jumpoffERP, result_xTFR, numareas)

# Store into Sheet 'Fertility' for further usage
row = 7
column = 4
for i in range(len(result_xTFR)):
    wb_wt_fertility.cell(row, column).value = result_xTFR[i]
    row += 1
wb_wt.save(wt_loc)

## PrepareData

#### Clear Input Data

In [13]:
# Before running the PrepareData Function, clear the input data if any
wb_wt_SmallAreaInputs = wb_wt["SmallAreaInputs"]
wb_wt_Accounts = wb_wt["Accounts"]

# Clear the data and save the writable workbook
clear_Input(wb_wt_SmallAreaInputs)
clear_Input(wb_wt_Accounts)
wb_wt.save(wt_loc)

#### Load in Required Data for Generating Input Data

In [14]:
# Read in readable required data for generating input data for further usage
sheet_label = wb_rd.sheet_by_name('Labels')
sheet_SmallAreaTotals = wb_rd.sheet_by_name('SmallAreaTotals')
sheet_agesex = wb_rd.sheet_by_name('AgeSexERPs')
sheet_mortality = wb_rd.sheet_by_name('Mortality')
sheet_migration = wb_rd.sheet_by_name('Migration')

# Temeorary read of new generated fertility
temp_wb_rd = xlrd.open_workbook(wt_loc)
sheet_fertility = temp_wb_rd.sheet_by_name('Fertility')

#### Start Creating Input Data

##### Define Variables

In [15]:
numages = 18                                    # Number of age groups in ERPs
lastage = 17                                    # Number of age groups in projections output
nLxMS = 21                                      # nLx mortality surface groups
age_ASFR = 7                                    # Age groups amount in ASFRs Model
numareas = int(sheet_label.cell_value(2256, 0)) # Number of small areas
final = int(sheet_label.cell_value(2271, 0))    # Final projection year number
SRB = sheet_fertility.cell_value(2270, 1)       # Assumed Sex Ratio at Birth (Male : Female)
modelTFR  = 0                                   # Model TFR Parameter
for i in range(2259, 2266): modelTFR += sheet_fertility.cell_value(i, 1)
modelTFR *= 5

##### Read in Data for Generating Input Data

In [16]:
## All two dimensional array with 0 and 1 as sex index, we treat 0 as female and 1 as male
# Read in AreaCode, AreaName, AgeGroup, Period-Cohort Labels
Areacode, Areaname, agelabel, pclabel = readCNAPC(numareas, numages, lastage, sheet_label)

# Read in Sex, Year, Interval Labels
sexlabel, yearlabel, intervallabel = readSYI(final, sheet_label)

# Read in Estimated & Projected Small Area Total Population (Aggregated)
TotPop = readSATP(2, sheet_SmallAreaTotals, numareas)

# Read in small area ERP (Estimated Resident Population) by age & sex (before and at jumpoff year)
ERP = readERP(2, numareas, numages, sheet_agesex)

# Area-specific TFRs
TFR = readTFR(final, numareas, sheet_fertility)

# Read in data for ASFRs and Preliminary ASFRs (Area specific Fertility Rate)
modelASFR, prelimASFR = readASFR(numareas, sheet_fertility, age_ASFR)

# Read in data for Life Expectancy and Mortality Surface
eO, MS_nLx, MS_TO = readLEMS(final, numareas, lastage, nLxMS, sheet_mortality)

# Read in data of migration data and for ASMRs (Area specific Mortality Rate)
totmig, modelASMR = readMASMR(numareas, lastage, sheet_migration)

##### Some Checks

In [17]:
# TFRs > 0
for i in range(final + 1):
    for a in range(numareas):
        if (TFR[i, a] < 0.1):
            print("Error: Year = ", yearlabel[i], " & Area = ", Areaname[a])
            break
    break

# eOs > 0
for i in range(final + 1):
    for a in range(numareas):
        for q in range(2):
            if (eO[i, a, q] < 0.1):
                print("Error: Year = ", yearlabel[i], " & Area = ", Areaname[a], " & Sex = ", sexlabel[q])
                break
        break
    break

# Check Migration Turnover > 0
for i in range(numareas):
    if (totmig[i] < 0.1):
        print("Error: Area =", Areaname[i])
    break
    
# Check Small Area Population Total > 0
for i in range(2):
    for a in range(numareas):
        if (TotPop[i, a] < 0.1):
            print("Error Year =", yearlabel[i], " & Area = ", Areaname[a])
        break
    break

##### Calculate Input Data

In [202]:
# Create input data for ASFR (Area specific Fertility Rate)
ASFR = inputASFR(final, numareas, age_ASFR, sheet_fertility, prelimASFR, modelASFR, TFR, modelTFR)

# Estimate base period birth by sex for the population 'Accounts'
tempbirthssex, temptotbirths = inputbirth(numareas, age_ASFR, ASFR, ERP, SRB)

# Calculate base period projected age-specific death rate (used in estimating death account)
ASDR = inputASDR(final, numareas, lastage, nLxMS, eO, MS_TO, MS_nLx)

# Calculate estimated death for age-sex cohorts and total population accounts
tempdeaths, temptotdeaths = inputdeath(numareas, lastage, ASDR, ERP)


In [203]:
def inputMigration(final, numareas, lastage, modelASMR, ERP, totmig, TotPop, temptotdeaths, temptotbirths, tempdeaths, tempbirthssex, ASDR):

    # Create empty array for storing migration related data
    inward = numpy.zeros((final + 1, numareas, 2, lastage + 1))
    ASOMR = numpy.zeros((final + 1, numareas, 2, lastage + 1))
    prelimmig = numpy.zeros((numareas, 2, lastage + 1))
    scaledmig = numpy.zeros((numareas, 2, lastage + 1))
    prelimbaseinward = numpy.zeros((numareas, 2, lastage + 1))
    prelimbaseoutward = numpy.zeros((numareas, 2, lastage + 1))
    cohortnetmig = numpy.zeros((numareas, 2, lastage + 1))
    adjnetmig = numpy.zeros((numareas, 2, lastage + 1))
    scalingfactor2a = numpy.zeros((numareas, 2, lastage + 1))
    scalingfactor2b = numpy.zeros((numareas, 2, lastage + 1))
    outward = numpy.zeros((numareas, 2, lastage + 1))

    # Collecting Migration Input Data
    for i in range(numareas):

        # Generate preliminary migration turnover by sex-age cohort based on model rates
        for s in range(2):
            # Generate preliminary migration turnover for all sex-age cohorts exclude 0-4, 85+
            for pc in range(1, lastage):
                prelimmig[i, s, pc] = modelASMR[s, pc] * 2.5 * (ERP[0, i, s, pc - 1] + ERP[1, i, s, pc])

            # Generate preliminary migration turnover for 0-4
            prelimmig[i, s, 0] = modelASMR[s, 0] * 2.5 * ERP[1, i, s, 0]

            # Generate preliminary migration turnover for 85+
            prelimmig[i, s, lastage] = modelASMR[s, lastage] * 2.5 * (ERP[0, i, s, lastage - 1] + ERP[0, i, s, lastage] + ERP[1, i, s, lastage])

        # Generate total preliminarymigration turnover value
        totprelimmig = 0
        for s in range(2):
            for pc in range(lastage + 1):
                totprelimmig += prelimmig[i, s, pc]
        
        # Collecting scaled preliminary migration by sex-age cohort to migration turnover 
        # as calculated by the crude migration turnover rate; and then divide by 2 to give the initial inward and outward migration
        for s in range(2):
            for pc in range(lastage + 1):
                scaledmig[i, s, pc] = prelimmig[i, s, pc] * (0.5 * totmig[i]) / totprelimmig

        # Calculate residual net migration for base period in each area i
        basenetmig = TotPop[1, i] - TotPop[0, i] + temptotdeaths[i] - temptotbirths[i]

        # Calculate the scaling factor to estimate separate inward and outward migration totals which are consistent with residual net migration
        scalingfactor1 = (basenetmig + (basenetmig ** 2 + (4 * 0.5 * totmig[i] * 0.5 * totmig[i])) ** 0.5) / (2 * 0.5 * totmig[i])

        # Calculate preliminary inward and outward migration by sex-age cohort (Different with the Original Value in 'Account')
        for s in range(2):
            for pc in range(lastage + 1):
                prelimbaseinward[i, s, pc] = scaledmig[i, s, pc] * scalingfactor1
                prelimbaseoutward[i, s, pc] = scaledmig[i, s, pc] / scalingfactor1

        # Calculate cohort-specific residual net migration (for adjustment usage)
        for s in range(2):
            # Generate cohort-specific residual net migration for all sex-age cohorts exclude 0-4, 85+
            for pc in range(1, lastage):
                cohortnetmig[i, s, pc] = ERP[1, i, s, pc] - ERP[0, i, s, pc - 1] + tempdeaths[i, s, pc]
            
            # Generate cohort-specific residual net migration for 0-4
            cohortnetmig[i, s, 0] = ERP[1, i, s, 0] - tempbirthssex[i, s] + tempdeaths[i, s, 0]

            # Generate cohort-specific residual net migration for 85+
            cohortnetmig[i, s, lastage] = ERP[1, i, s, lastage] - (ERP[0, i, s, lastage - 1] + ERP[0, i, s, lastage]) + tempdeaths[i, s, lastage]

        # Adjusted cohort-specific net migration with averaged over sex for selected ages
        for s in range(2):
            for pc in range(14):
                adjnetmig[i, s, pc] = 0.5 * (cohortnetmig[i, 0, pc] + cohortnetmig[i, 1, pc])
            for pc in range(14, lastage + 1):
                adjnetmig[i, s, pc] = cohortnetmig[i, s, pc]
        
        # Calculate scaling factors to adjust directional migration (Raw & Smoothed)

In [204]:
test1 = inputMigration(final, numareas, lastage, modelASMR, ERP, totmig, TotPop, temptotdeaths, temptotbirths, tempdeaths, tempbirthssex, ASDR)

In [ ]:
test1